In [81]:
import tensorflow as tf
import time
import numpy as np

In [82]:
# Parameters
learning_rate = 0.1
training_epochs = 20
batch_size = 40
display_step = 1

n_samples_tr = 5000
n_samples_te = 1000

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_hidden_1 = 24 # 1st layer number of features
n_hidden_2 = 12 # 2nd layer number of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [83]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

ds_tr = mnist.train
xs_tr = ds_tr.images[:n_samples_tr]
ys_tr = ds_tr.labels[:n_samples_tr]

ds_te = mnist.test
xs_te = ds_te.images[:n_samples_te]
ys_te = ds_te.labels[:n_samples_te]

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [84]:
def build_weights(nb_input, nb_hidden):
    return tf.Variable(tf.random_normal([nb_input, nb_hidden]) / np.sqrt(n_hidden_1))

In [85]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer

# Store layers weight & bias
weights = {
    'h1': build_weights(n_input, n_hidden_1),
    'h2': build_weights(n_hidden_1, n_hidden_2),
    'out': build_weights(n_hidden_2, n_classes)
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# mse, normalized by class amount
cost = tf.reduce_mean(tf.square(tf.subtract(pred, y)) * np.square(n_classes))

# tf.reduce_mean(tf.squared_difference(py_x, Y))

# Define loss and optimizer
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)



In [86]:
# Initializing the variables
init = tf.global_variables_initializer()

time_start = time.time()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training cycle
    for epoch in range(training_epochs):
        
        avg_cost = 0.
        
        for i in range(0, xs_tr.shape[0], batch_size):
            
            batch_x = xs_tr[i : i + batch_size]
            batch_y = ys_tr[i : i + batch_size]
            
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / n_samples_tr
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch: {}, cost = {}".format((epoch+1), avg_cost)
            
            correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
            
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            print "Accuracy:", accuracy.eval({x: xs_te, y: ys_te})
            
    print "Optimization Finished!"
    
    
time_end = time.time()
print "Training took {} seconds.".format(time_end - time_start)

Epoch: 1, cost = 0.233728190231
Accuracy: 0.179
Epoch: 2, cost = 0.221622480392
Accuracy: 0.285
Epoch: 3, cost = 0.203262624168
Accuracy: 0.441
Epoch: 4, cost = 0.160405083466
Accuracy: 0.641
Epoch: 5, cost = 0.12180510273
Accuracy: 0.739
Epoch: 6, cost = 0.0968593564987
Accuracy: 0.774
Epoch: 7, cost = 0.0783428847313
Accuracy: 0.794
Epoch: 8, cost = 0.0661811619759
Accuracy: 0.824
Epoch: 9, cost = 0.0567911498427
Accuracy: 0.841
Epoch: 10, cost = 0.0490834114313
Accuracy: 0.852
Epoch: 11, cost = 0.0433296534777
Accuracy: 0.863
Epoch: 12, cost = 0.0390944684327
Accuracy: 0.864
Epoch: 13, cost = 0.0357953491867
Accuracy: 0.871
Epoch: 14, cost = 0.0331024745524
Accuracy: 0.874
Epoch: 15, cost = 0.0308211925507
Accuracy: 0.878
Epoch: 16, cost = 0.0288353870571
Accuracy: 0.884
Epoch: 17, cost = 0.0270923203915
Accuracy: 0.886
Epoch: 18, cost = 0.0255525894672
Accuracy: 0.891
Epoch: 19, cost = 0.0241775598943
Accuracy: 0.894
Epoch: 20, cost = 0.0229327447534
Accuracy: 0.895
Optimization Fi